# House Prices Kaggle Competition

This notebook simply explores the dataset to see what insights can be gained.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Import and inspect the data

In [ ]:
df = pd.read_csv('../data/train.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
featuresWithNullValues = df.isnull().sum()
print(featuresWithNullValues[featuresWithNullValues > 0])

# Dealing with Missing Data

The **train.csv** dataset has 1460 records and 81 features. There are 19 features with missing data that need to be treated with a variety of strategies. Ranked in order of most missing data to least:
- PoolQC = 99.52%
- MiscFeature = 96.30%
- Alley = 93.76%
- Fence = 80.75%
- FireplaceQu = 47.26%
- LotFrontage = 17.74%
- GaragType, GarageYrBlt, GarageFinish, GarageQual, GarageCond = 5.55%
- BsmtExposure, BsmtFinType2 = 2.60%
- BsmtQual, BsmtCond, BsmtFinType1 = 2.53%
- MasVnrType, MasVnrArea = 0.55%
- Electrical = 0.07%

### PoolQC (99.52%)

PoolQC (Pool quality) is related to PoolArea which is not missing data and has a lot of 0's, which likely means there is no pool. Let's see if missing PoolQC values are correlated to PoolArea values that are 0.

In [ ]:
df.PoolArea.value_counts().head(3)

There are 1453 PoolArea values equal to 0 which matches the number of missing PoolQC values. Are they 100% correlated?

In [ ]:
len(df[(df.PoolArea==0) & df.PoolQC.isnull()])

Yes they are. The data description allows for 'NA' as an option. **Let's replace nan with NA.**

In [ ]:
df.PoolQC.fillna('NA', inplace=True)

### MiscFeatures (96.30%)

Miscellaneous feature not covered in other categories. There is a MiscVal feature that is not missing data. I am guessing everywhere there is missing data for MiscFeature there will be a $0 value for MiscVal.

In [ ]:
df.MiscVal.value_counts().head(3)

In [ ]:
df.MiscFeature.value_counts()

There are 2 more 0 value features than missing misc features. Is this because 'Othr' miscellaneous feature is 0 value?

In [ ]:
temp_df = df[['MiscFeature', 'MiscVal']]

shed_df = temp_df[temp_df['MiscFeature'] == 'Shed'] 
gar2_df = temp_df[temp_df['MiscFeature'] == 'Gar2'] 
othr_df = temp_df[temp_df['MiscFeature'] == 'Othr'] 
tenc_df = temp_df[temp_df['MiscFeature'] == 'TenC'] 

othr_df.head()

No, one of the sheds (index 1200) is 0 as well.

I'm not going to worry about the additional 2. If MiscFeature is nan **I will set to 'NA'** as defined in the data description.

In [ ]:
df.MiscFeature.fillna('NA', inplace=True)

### Alley (93.76%)

Type of alley access to property

- Grvl	Gravel
- Pave	Paved
- NA 	No alley access

Some possibly related features:

- Utilities: Type of utilities available - limited utilities access might imply no alley access.
- BldgType: Type of dwelling - townhouses likely have no alley access.

In [ ]:
df.Utilities.value_counts()

That wasn't helpful, what about BldgType?

In [ ]:
df.BldgType.value_counts()

Nothing there either. How about the Neighborhood?

In [ ]:
df.Neighborhood.value_counts()

In [ ]:
#len(df[(df.PoolArea==0) & df.PoolQC.isnull()])
temp_df = df[df['Alley'].notnull()] 
temp_df.Neighborhood.value_counts()

I don't see any relationships that can help impute the missing values. Given almost 94% of the data is missing, **I will just drop this feature.**

In [ ]:
df.drop('Alley', axis=1, inplace=True)

### Fence (80.75%)

Fence quality
		
- GdPrv	Good Privacy
- MnPrv	Minimum Privacy
- GdWo	Good Wood
- MnWw	Minimum Wood/Wire
- NA	No Fence

In [ ]:
df.Fence.value_counts()

There are too many missing values and no other feature gives a clue about this one. **So just drop it.**

In [ ]:
df.drop('Fence', axis=1, inplace=True)

### FireplaceQu (47.26%)

Fireplace quality

- Ex	Excellent - Exceptional Masonry Fireplace
- Gd	Good - Masonry Fireplace in main level
- TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
- Fa	Fair - Prefabricated Fireplace in basement
- Po	Poor - Ben Franklin Stove
- NA	No Fireplace

Fireplaces: Number of fireplaces

In [ ]:
df.Fireplaces.value_counts()

There are 690 homes with 0 fireplaces. This matches the number of missing FireplaceQu values.

In [ ]:
len(df[(df.Fireplaces==0) & df.FireplaceQu.isnull()])

All I need to do is **fill nan's with 'NA'.**

In [ ]:
df.FireplaceQu.fillna('NA', inplace=True)

### LotFrontage (17.74%)

Linear feet of street connected to property.

Some related: 
- LotConfig: Lot configuration

In [ ]:
df.LotFrontage.value_counts()

There are too many missing values and no other feature gives a clue about this one. **So just drop it.**

In [ ]:
df.drop('LotFrontage', axis=1, inplace=True)

### GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond (5.55%)

In [ ]:
temp_df = df[['GarageCars', 'GarageArea', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']]
temp_df[temp_df['GarageType'].isnull()]

In [ ]:
temp_df.GarageCars.value_counts().head(10)

There are 81 records for GarageType, GarageYrBlt, GarageFinish, GarageQual, and GarageCond. And there appears to be 81 correlated records for GarageCars and GarageArea with a value of 0.

**fill GarageYrBlt with 0 and all others with 'NA'**

In [ ]:
df.GarageType.fillna('NA', inplace=True)
df.GarageYrBlt.fillna(0, inplace=True)
df.GarageFinish.fillna('NA', inplace=True)
df.GarageQual.fillna('NA', inplace=True)
df.GarageCond.fillna('NA', inplace=True)

### BsmtQual, BsmtCond, BsmtFinType1 (2.53%) - BsmtExposure, BsmtFinType2 (2.60%)

- BsmtQual: Evaluates the height of the basement
- BsmtCond: Evaluates the general condition of the basement
- BsmtFinType1: Rating of basement finished area
- BsmtExposure: Refers to walkout or garden level walls
- BsmtFinType2: Rating of basement finished area (if multiple types)

TotalBsmtSF is not missing any values, could that be a predictor for the missing values above?

In [ ]:
df.TotalBsmtSF.value_counts().head(3)
#df.TotalBsmtSF.value_counts().sum()

There are 37 values with 0 sf which matches number of missing values for BsmtQual, BsmtCond, BsmtFinType1. Are they 100% correlated?

In [ ]:
len(df[(df.TotalBsmtSF==0) & df.BsmtQual.isnull() & df.BsmtCond.isnull() & df.BsmtFinType1.isnull()])

Yes they are. I can **set these missing values to 'NA'**

In [ ]:
df.BsmtQual.fillna('NA', inplace=True)
df.BsmtCond.fillna('NA', inplace=True)
df.BsmtFinType1.fillna('NA', inplace=True)

How many missing values for BsmtExposure and BsmtFinType2?

In [ ]:
1460 - df.BsmtExposure.value_counts().sum()
1460 - df.BsmtFinType2.value_counts().sum()

Is there correlation for sf?

In [ ]:
len(df[(df.TotalBsmtSF==0) & df.BsmtExposure.isnull() & df.BsmtFinType2.isnull()])

Yes there is. **Set these to 'NA'.**

In [ ]:
for i in df.index:
    if(df.at[i, 'TotalBsmtSF']==0):
        df.at[i, 'BsmtExposure'] = 'NA'
        df.at[i, 'BsmtFinType2'] = 'NA'

There is still one more record missing values BsmtExposure and BsmtFinType2. Are they correlated?

In [ ]:
temp_df = df[['TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtExposure', 'BsmtFinType2']]

temp_df[temp_df.BsmtExposure.isnull() | temp_df.BsmtFinType2.isnull()]

No, lets look at value counts for each and choose the most frequent option.

In [ ]:
df.BsmtExposure.value_counts()

In [ ]:
df.BsmtFinType2.value_counts()

**Set the missing values**

In [ ]:
index = df.index[df.BsmtExposure.isnull()]
df.at[index[0], 'BsmtExposure'] = 'No'

In [ ]:
index = df.index[df.BsmtFinType2.isnull()]
df.at[index[0], 'BsmtFinType2'] = 'Unf'

### MasVnrType, MasVnrArea (0.55%)

Are all the same records missing these or are missing values spread around?

In [ ]:
len(df[df.MasVnrType.isnull() & df.MasVnrArea.isnull()])

All 8 missing values overlap. Exterior1st and Exterior2nd tell us what veneer type to assign to null values. Will have to take average area.

In [ ]:
df.MasVnrArea.value_counts()

An example of the Exterior1st and Exterior2nd values when veneer type and area are null.

In [ ]:
temp_df = df[['Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea']]

temp_df[temp_df.MasVnrArea.isnull() & temp_df.MasVnrType.isnull()]

A conditional test can be used.

In [ ]:
# calculate mean of all MasVnrArea values that are greater than 0.
# for each house in data frame:
#     if Exterior1st or Exterior2nd = Brick Common, then:
#         MasVnrType = BrkCmn
#         MasVnrArea = (mean)
#     else if Exterior1st or Exterior2nd = Brick Face, then:
#         MasVnrType = BrkFace
#         MasVnrArea = (mean)
#     else if Exterior1st or Exterior2nd = Cinder Block, then:
#         MasVnrType = CBlock
#         MasVnrArea = (mean)
#     else if Exterior1st or Exterior2nd = Stone, then:
#         MasVnrType = Stone
#         MasVnrArea = (mean)
#     else:
#         MasVnrType = None
#         MasVnrArea = 0.0

In [ ]:
temp_df = df[(df['MasVnrArea'] > 0.0)]
area_mean = temp_df.MasVnrArea.mean()
area_mean

**Fill in missing values**

In [ ]:
for i in df.index:
    value = df.at[i, 'MasVnrType']
    if(pd.isnull(value)):
        if((df.at[i, 'Exterior1st'] == 'Brick Common') or (df.at[i, 'Exterior2nd'] == 'Brick Common')):
            print('Found Brick Common')
            df.at[i, 'MasVnrType'] = 'BrkCmn'
            df.at[i, 'MasVnrArea'] = area_mean
        elif((df.at[i, 'Exterior1st'] == 'Brick Face') or (df.at[i, 'Exterior2nd'] == 'Brick Face')):
            print('Found Brick Face')
            df.at[i, 'MasVnrType'] = 'BrkFace'
            df.at[i, 'MasVnrArea'] = area_mean
        elif((df.at[i, 'Exterior1st'] == 'Cinder Block') or (df.at[i, 'Exterior2nd'] == 'Cinder Block')):
            print('Found Cinder Block')
            df.at[i, 'MasVnrType'] = 'CBlock'
            df.at[i, 'MasVnrArea'] = area_mean
        elif((df.at[i, 'Exterior1st'] == 'Stone') or (df.at[i, 'Exterior2nd'] == 'Stone')):
            print('Found Stone')
            df.at[i, 'MasVnrType'] = 'Stone'
            df.at[i, 'MasVnrArea'] = area_mean
        else:
            print('Found one of the others')
            df.at[i, 'MasVnrType'] = 'None'
            df.at[i, 'MasVnrArea'] = 0.0
    

Verify that there are no longer any nan values for veneer area and type

In [ ]:
temp_df = df[['Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea']]

temp_df[temp_df.MasVnrArea.isnull() & temp_df.MasVnrType.isnull()]

### Electrical (0.07%)




In [ ]:
df.Electrical.value_counts()

There is only one missing value and 91.37% of values are SBrkr, so just **set any missing values to SBrkr.**

In [ ]:
df.Electrical.fillna('SBrkr', inplace=True)

### Let's see how we are doing.

In [ ]:
featuresWithNullValues = df.isnull().sum()
print(featuresWithNullValues[featuresWithNullValues > 0])